Coins: a Primer on Currency
===

I've always been curious: who created the coins?

Human are more proficient with decimal numbers, instead of, say, binary or hexadecimal, that much is understandable. Just look at the number of fingers and toes you have. Then it is natural to have a 5-cent coin and a 1-cent coin.

But where do the rest come from?

Who decided that we don't need 3-cent, 4-cent, 7-cent coins, etc...? Even a two-cent coin is not strictly required. In the United States, for instance, they don't weigh in their [two-cents](https://www.urbandictionary.com/define.php?term=2%20cents).

From 1 to 100 cents, here is what economies around the world have created:

- [States](https://en.wikipedia.org/wiki/Coins_of_the_United_States_dollar): 1, 5, 10, 25, 50
- [Europe](https://en.wikipedia.org/wiki/Euro_coins): 1, 2, 5, 10, 20, 50
- Arbitrarily, one might want to have: 1, 5, 10, 20, 50.

Over 1 dollar, you would have the same series of numbers repeated again, so let's restrict ourselves to the sub-dollar coins.

Naturally, the question is: which system is more efficient? Specifically, assume that you have an amount less than 100 cents to pay, and you still have the privilege to pay with coins, then what is the minimum number of coins you have to pay, in each system?

Whoever added the quarter (25-cent) coin in the US could be pretty proud. When you have to pay 25 cents, but you don't have the exact 25-cent coin, in the US, you would need 3 coins: 10 + 10 + 5 = 25 cents. In Europe, you need only two: 20 + 5 = 25 cents.

When you need to pay 75 cents though, American needs a minimum of two coins: 50 + 25 = 75 cents, while a European needs at least 3.

How does this generalize? Over the whole spectrum of amounts from 1 to 99 cents, which one is more economically efficient? You might have noticed that, sub-dollar, there are 5 types of coins in the US, while Europe has 6. How does this effect the minimum number of coins you need for a given amount?

With a little piece of code, let's work this out. I may not be qualified for a Nobel for this, but I got your attention.

Problem Statement
---

The first question is: given an amount less than 100 cents, how many ways are there to pay, in each of the systems mentioned above?

We will assume there is an indefinite amount of coins for each type. This usually doesn't hold in practice for a specific consumer, but from the perspective of the people who design the system, it sounds reasonable.

Doing this is not difficult, here is a recursive implementation:

In [1]:
def cents(values, amount, solutions):
    if amount == 0:
        solutions.append([0] * len(values))
    elif len(values) == 1:
        max_cnt = amount // values[0]
        if amount == max_cnt * values[0]:
            solutions.append([max_cnt])
    else:
        max_cnt = amount // values[0]
        for i in range(max_cnt, -1, -1):
            remaining = amount - (values[0] * i)
            sub_solutions = []
            cents(values[1:], remaining, sub_solutions)
            if len(sub_solutions) > 0:
                for s in sub_solutions:
                    solutions.append([i] + s)

The diffrent ways to pay for 31 cents in the US is:

In [9]:
# Intentionally descendantly sorted. May save a few iterations.
us_coins = [50, 25, 10, 5, 1]

solutions = []
cents(us_coins, 31, solutions)
counts = [sum(_) for _ in solutions]

# Don't mind me. This is a non-Numpy implementation. No sorted() too.
import functools
mininum_coin = functools.reduce(lambda a, b: a if a[0] < b[0] else b, zip(counts, solutions))[1]

print('{} ways to pay'.format(len(solutions)))
print('The most economical way is: {}'.format(mininum_coin))
solutions

18 ways to pay
The most economical way is: [0, 1, 0, 1, 1]


[[0, 1, 0, 1, 1],
 [0, 1, 0, 0, 6],
 [0, 0, 3, 0, 1],
 [0, 0, 2, 2, 1],
 [0, 0, 2, 1, 6],
 [0, 0, 2, 0, 11],
 [0, 0, 1, 4, 1],
 [0, 0, 1, 3, 6],
 [0, 0, 1, 2, 11],
 [0, 0, 1, 1, 16],
 [0, 0, 1, 0, 21],
 [0, 0, 0, 6, 1],
 [0, 0, 0, 5, 6],
 [0, 0, 0, 4, 11],
 [0, 0, 0, 3, 16],
 [0, 0, 0, 2, 21],
 [0, 0, 0, 1, 26],
 [0, 0, 0, 0, 31]]

The most economical way to pay 31 cents is with 3 coins: 25 + 5 + 1.

In Europe:

In [11]:
eu_coins = [50, 20, 10, 5, 2, 1]

solutions = []
cents(eu_coins, 31, solutions)
counts = [sum(_) for _ in solutions]

mininum_coin = functools.reduce(lambda a, b: a if a[0] < b[0] else b, zip(counts, solutions))[1]

print('{} ways to pay'.format(len(solutions)))
print('The most economical way is: {}'.format(mininum_coin))

116 ways to pay
The most economical way is: [0, 1, 1, 0, 0, 1]


Unsurprisingly, 20 + 10 + 1 = 31 cents, but Europeans have way more ways to pay.

You might have noticed how a little functional programming was used. For the sake of doing this, here is a one-liner functional implementation of the same `cents()` function.

In [13]:
import itertools

cents2 = lambda values, amount: list(filter(lambda e: e[1] == amount, map(
    lambda l: (l, sum(map(lambda b: b[0] * b[1], zip(l, values)))), 
    itertools.product(*map(lambda x: list(range(1 + (amount // x))), values)))))

# Or, more efficiently
cents2 = lambda values, amount: list(filter(lambda l: sum(map(lambda b: b[0] * b[1], zip(l, values))) == amount,
                       itertools.product(*map(lambda x: list(range(1 + (amount // x))), values))))

We are all set for the fun part: how does that generalize to every amount, from 1 to 99 cents?

Over the spectrum
---

In [14]:
coins = [('EU', [50, 20, 10, 5, 2, 1]), ('US', [50, 25, 10, 5, 1]), ('Another', [50, 20, 10, 5, 1])]
ways = {'EU': [], 'US': [], 'Another': []}
min_way = {'EU': [], 'US': [], 'Another': []}
avg_way = {'EU': [], 'US': [], 'Another': []}

for amount in range(1, 100):
    for lb, vals in coins:
        s = []
        cents(vals, amount, s)
        counts = [sum(a) for a in s]
        
        # changed_min_cnt = min(counts) if 1 not in counts else min(filter(lambda _: _ != 1, counts), default=1)
        # median: sorted(counts)[ways // 2]
        ways[lb].append(len(counts))
        min_way[lb].append(min(counts))
        avg_way[lb].append(float(sum(counts)) / len(counts))

In [46]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, show

output_notebook()

def plot(data, title='', y_range=-1):
    x_range = list(range(1, 100))
    if y_range == -1:
        p = figure(x_range=(1, 100), title=title, height=350, width=950)
    else:
        p = figure(x_range=(1, 100), y_range=(0, y_range), title=title, height=350, width=950)

    lefts = [-0.25, 0, 0.25]
    colors = ['#859581', '#718dbf', '#e84d60']
    for i, item in enumerate(data.items()):
        k, v = item
        p.vbar(x=[x + lefts[i] for x in x_range], width=0.25, bottom=0,
               top=v, color=colors[i], legend_label=k)

    # p.x_range.range_padding = 0.1
    p.xgrid.grid_line_color = None
    p.legend.location = "top_left"
    p.legend.orientation = "horizontal"

    show(p)

Loading BokehJS ...

Number of combinations
---

- With 6 types of coins, Europeans tend to have more ways to pay.
- With the same 5 types of coins, Americas, with the quarter coin, has less choices than the system with the 20-cent coin.

In [47]:
plot(ways, title='Number of combinations', y_range=500)

Minimum number of coins needed
---

Becase of the penny (1-cent), the maximum count is unsurprisingly always equal to the amount. Here we look at the minimum count.

As I examine the following plot, Uncle Sam was whispering to my ears: "Hmm, not too bad, I'd say."

As a perk of being economical (making only 5 types of coins), the US system needs 7 coins to pay for 74 cents (`50 + 2*10 + 4*1`), while the EU system only needs 4 coins (`50 + 20 + 2 * 2`).

In [48]:
plot(min_way, 'Minimum count')

Among the two systems with 5 types of coins, there are 20 cases where Americas need more coins, 59 cases where they are the same, and 20 cases the US system needs less coins, as shown below:

In [63]:
five_types = list(zip(min_way['US'], min_way['Another']))
mores = list(filter(lambda a: a[0] > a[1], five_types))
print('More: {} cases of total {}'.format(len(mores), sum(map(lambda a: a[0] - a[1], mores))))
equals = filter(lambda a: a[0] == a[1], five_types)
print('Equal: {} cases'.format(len(list(equals))))
lesses = list(filter(lambda a: a[0] < a[1], five_types))
print('Less: {} cases of total {}'.format(len(lesses), sum(map(lambda a: a[1] - a[0], lesses))))

More: 20 cases of total 20
Equal: 59 cases
Less: 20 cases of total 20


Average number of coins needed
---

Among all the combinations, what is the average number of coins needed to pay for a given amount?

In [55]:
plot(avg_way, title='Average number of coins')

Regardless of the math, if one fine morning, you go to Starbucks with the bills in hand, and they return you with four pennies, you will feel the ache of not having two-cents.

Extra
===

The next natural question is: what if we have `[1, 3, 7, 10, 30, 70]`-cent coins? What is the most optimal sequence of values?

Mathematicans might come up with an elegant solution. Economically speaking, the point is to reduce the coin types while maintaining a reasonable efficiency.

So here are a few things to try out:

In [76]:
# The average of `the minimum numbers of coins needed to pay` every amount from 1 to 99 cents

def avg_min(values):
    mins = []
    for amount in range(1, 100):
        solutions = []
        cents(values, amount, solutions)
        mins.append(min(map(sum, solutions)))
    return float(sum(mins)) / len(mins)

In [75]:
print(avg_min([50, 20, 10, 5, 2, 1]))
print(avg_min([60, 30, 10, 6, 3, 1]))
print(avg_min([65, 30, 10, 7, 3, 1]))

print(avg_min([50, 25, 10, 5, 1]))
print(avg_min([50, 25, 5, 3, 1]))
print(avg_min([50, 30, 5, 3, 1]))

print(avg_min([50, 20, 5, 1]))
print(avg_min([50, 30, 5, 1]))

3.4343434343434343
3.2525252525252526
3.202020202020202
4.242424242424242
4.242424242424242
3.9393939393939394
4.646464646464646
4.747474747474747


Among the 6-coins options, `[6, 3, 1]` seems to have a chance to be better than `[5, 2, 1]`, which is the most common scheme. I would prefer `[7, 3, 1]` though, because for 6 cents, you may have 2 3-cent coins, which is economical enough.

If your hypothetical kingdom opts for 5-coins scheme, maybe consider `[50, 30, 5, 3, 1]`, which, in average, is more efficient than the US coin scale.

For coins alone, I am fully aware that this study came too late. If it was conducted back in the early 1900s, where people were buying loafs of bread with tiny coins, it would've been much more helpful.

But let's keep in mind that these findings also apply to the bills...

I took the liberty to go ahead and provide a script to run Bayesian Optimization to find the best values for each type of coins. You might need an account at some certain important location to run the following:

In [ ]:
# FIX: you will need to change the cents() function to take into account fractional values.
# Why? It's fun to have a 5.83758603-cent coin, isn't it?
# Funnier if you let it go negative...

def evaluate_model(params):
    vals = sorted([params['x1'], params['x2'], params['x3'], params['x4']], reverse=True)
    cnts = 0
    sums = 0.
    print('Evaluating {}'.format(vals))
    for i in range(1, 100):
        s = []
        cents(vals, i, s)
        if len(s) > 0:
            counts = [sum(a) for a in s]
            sums += min(counts)
            cnts += 1
    if cnts < 99:
        print('Infeasible set of values: {}'.format(vals))
        return 10E6   # math.inf
    return sums / cnts


def optimize():
    import sigopt

    sigopt.set_project("cents")

    experiment = sigopt.create_experiment(
        name='Coins Optimization',
        parameters=[
            dict(name='x1', type='double', bounds=dict(min=0.00001, max=99.99999)),
            dict(name='x2', type='double', bounds=dict(min=0.00001, max=99.99999)),
            dict(name='x3', type='double', bounds=dict(min=0.00001, max=99.99999)),
            dict(name='x4', type='double', bounds=dict(min=0.00001, max=99.99999)),
        ],
        metrics=[dict(name='avg_minimum_required', objective='minimize')],
        parallel_bandwidth=1,
        # Define a budget for your experiment
        budget=100,
    )
    print("Created experiment: https://app.sigopt.com/experiment/" + experiment.id)

    for run in experiment.loop():
        with run:
            value = evaluate_model(run.params)
            run.log_metric("avg_minimum_required", value)

    # Fetch the best configuration and explore your experiment
    best_run = next(experiment.get_best_runs())
    best_assignments = best_run.assignments
    print("Best Assignments: " + str(best_assignments))
    print("Explore your experiment: https://app.sigopt.com/experiment/" + experiment.id + "/analysis")


if __name__ == '__main__':
    optimize()

Little disclaimer
---

Although I'm unemployed, these findings are in no way politically biased. Unless the tools I am using, which include a compiler, a browser, and all the turtles all the way down, are politically biased, these findings are neutral.
